<a href="https://colab.research.google.com/github/sokrypton/roscon2024/blob/main/SaProt_Tutorial_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/westlake-repl/SaprotHub.git

import os
# Check whether the server is local or from google cloud
root_dir = os.getcwd()

import sys
sys.path.append(f"{root_dir}/SaprotHub")
import saprot
print("SaProt is installed successfully!")
os.system(f"chmod +x {root_dir}/SaprotHub/bin/*")
import pandas as pd
from saprot.utils.foldseek_util import get_struc_seq
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import gdown

FOLDSEEK_PATH = Path(f"{root_dir}/SaprotHub/bin/foldseek")

In [ ]:
structure_url = "https://drive.google.com/file/d/1fkOrLdYEAw67pIyjJh0W0jRNhhicIvHY/view?usp=drive_link"
structure_output_path = '/content/AF2_predicted_best_structures.zip'
gdown.download(structure_url, structure_output_path, quiet=False,fuzzy=True)

In [ ]:
sequence_url = "https://drive.google.com/file/d/1AkHwAKl-S-lyb2C5GANPOx0UMqvQ-PIE/view?usp=drive_link"
sequence_output_path = '/content/thermomut_all.csv'
gdown.download(sequence_url, sequence_output_path, quiet=False,fuzzy=True)

In [ ]:
!unzip /content/AF2_predicted_best_structures.zip -d /content/SaprotHub/

In [ ]:
def get_sa(directory, name, mut_type, position, wild_type, mutation, minus_1= False, chain='A'):
    pdbs=directory
    index = name.find(".pdb") + len(".pdb")
    name = name[:index]
    pdb_file = os.path.join(pdbs, name)
    process_id = os.getpid()
    struct_seq_data = get_struc_seq(FOLDSEEK_PATH, pdb_file, [chain], process_id)

    if chain not in struct_seq_data:
        print(f"Chain {chain} not found in structure sequence data for", name)
        return None

    parsed_seqs = struct_seq_data[chain]
    seq, foldseek_seq, combined_seq = parsed_seqs

    try:

        if  mut_type ==True:
            return combined_seq
        else:
            if minus_1 ==True:
                pdb_idx = int(position)-1
            else:
                pdb_idx = int(position)
            assert combined_seq[pdb_idx*2] == wild_type
            combined_seq_list = list(combined_seq)
            combined_seq_list[pdb_idx*2] = mutation
            combined_seq= ''.join(combined_seq_list)

            return combined_seq
    except:
        print(os.path.join(pdbs, name))
        return None


In [ ]:
df_training = pd.read_csv('/content/thermomut_all.csv')
df_training = df_training.rename(columns={'aa_seq': 'Sequence', 'ddG': 'label'})
df_training['stage'] = 'test'
tqdm.pandas(desc="Processing SA data")

df_training['sa'] = df_training.progress_apply(
    lambda row: get_sa('/content/SaprotHub/AF2_predicted_best_structures', row['name'], row['WT'], row['position'], row['wild_type'], row['mutation'],  minus_1 =False),
    axis=1
)

df_training = df_training.drop(columns=['Sequence'])
df_training = df_training.rename(columns={'sa': 'Sequence'})
df_training = df_training[['name', 'Sequence', 'label', 'stage']]
df_training.to_csv('/content/thermomut_all_ddg_test.csv')